In [1]:
import torch
from torch import nn
import torchvision
from torchvision import transforms

from utils import *
from learner import *

In [ ]:
BATCH_SIZE = 64
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_dataloader, test_dataloader, class_names = dataloader(train_dataset, test_dataset, BATCH_SIZE)

In [ ]:
def noisify(x0, alpha_bar):
    device = x0.device
    n = len(x0)
    t = torch.randint(0, n_steps, (n,), dtype=torch.long)
    noise = torch.randn(x0.shape, device=device)
    alpha_bar_t = alpha_bar[t].reshape(-1, 1, 1, 1).to(device)
    xt = alpha_bar_t.sqrt()*x0 + (1-alpha_bar_t).sqrt()*noise
    return (xt, t.to(device)), noise

In [ ]:
class DDPM():
    def __init__(self, beta_min, beta_max, n_steps):
        init_attr(self, locals=locals())
        self.beta = torch.linspace(self.beta_min, self.beta_max, self.n_steps)
        self.alpha = 1 - self.beta
        self.alpha_bar = self.alpha.cumprod(dim=0)
        self.sigma = self.beta.sqrt()